In [ ]:
# thesis or cyrille's paper?
# THESIS = True
THESIS = False
fancyblue = (104/255, 171/255, 219/255)
fancyred = (254/255, 110/255, 99/255)
fancypurple = (211/255, 121/255, 251/255)

In [ ]:
import colorsys
import matplotlib.ticker as ticker
# Define the RGB colors
fancyblue = (104/255, 171/255, 219/255)
fancyred = (254/255, 110/255, 99/255)
fancypurple = (211/255, 121/255, 251/255)

# Convert RGB to HSV
fancyblue_hsv = colorsys.rgb_to_hsv(*fancyblue)
fancyred_hsv = colorsys.rgb_to_hsv(*fancyred)
fancypurple_hsv = colorsys.rgb_to_hsv(*fancypurple)

# Average the saturation and value
average_saturation = (fancyblue_hsv[1] + fancyred_hsv[1] + fancypurple_hsv[1]) / 3
average_value = (fancyblue_hsv[2] + fancyred_hsv[2] + fancypurple_hsv[2]) / 3

average_saturation /= 1.2
average_value = 1

# Create new colors by changing the hue
# For fancyorange, use a hue around 30 degrees (1/12 of the circle)
# For fancygreen, use a hue around 120 degrees (1/3 of the circle)
fancyorange_hue = 30 / 360
fancygreen_hue = 120 / 360

fancyorange_hsv = (fancyorange_hue, average_saturation, average_value)
fancygreen_hsv = (fancygreen_hue, average_saturation, average_value)

# Convert the new HSV colors back to RGB
fancyorange = colorsys.hsv_to_rgb(*fancyorange_hsv)
fancygreen = colorsys.hsv_to_rgb(*fancygreen_hsv)

# Make orange a bit brighter
# fancyorange = (1, fancyorange[1] * 1.15, fancyorange[2] * 1.15)

# Print the results
print("Fancy Orange RGB:", fancyorange)
print("Fancy Green RGB:", fancygreen)

# show the colors
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
ax.plot(np.random.rand(10), color=fancyblue)
ax.plot(np.random.rand(10), color=fancyred)
ax.plot(np.random.rand(10), color=fancypurple)
ax.plot(np.random.rand(10), color=fancyorange)
ax.plot(np.random.rand(10), color=fancygreen)
plt.show()
print(average_saturation, average_value)

In [ ]:
# fancygreen = (0/255, 200/255, 140/255)
# fancyorange = (255/255, 180/255, 0/255)
fancygray = (180/255, 180/255, 180/255)

fancycolors = [fancypurple, fancyorange, fancyred]

In [ ]:
# font size:
fs = 15

# data center power usage efficiency
PUE = 1.1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.constrained_layout.use'] = True
plt.tight_layout()

In [ ]:
df = pd.read_csv('../bitcoinforum/6_merging/monthly_stuff.csv')
df['date'] = pd.to_datetime(df['date'])

In [ ]:
seconds_in_block = 600
df['joules_per_coin'] = seconds_in_block * df['hashrate'] / (df['efficiency'] * df['coins_per_block'])
joules_per_coin = df[['date', 'joules_per_coin']]
df['joules_per_dollar'] = df['joules_per_coin'] / df['price']
df.set_index('date', inplace=True)
joules_per_dollar = df[['joules_per_dollar']]
joules_per_coin.index = joules_per_coin['date']
joules_per_coin = joules_per_coin.drop(columns=['date'])

In [ ]:
joules_per_coin.to_csv('../excel_sheet/joules_per_coin.csv')

In [ ]:
joules_per_second = df['hashrate'] / df['efficiency']
# reverse order
joules_per_second = joules_per_second.iloc[::-1]

In [ ]:
price = df['price']
log_price = np.log10(df['price'])
# log_price = df['price']
log_price = log_price.iloc[::-1]
log_price_normalized = (log_price - log_price.min()) / (log_price.max() - log_price.min())
log_price_normalized = log_price_normalized.rename('log_price')


halving_dates = pd.DataFrame({
    'date': ['2012-11-28', '2016-07-09', '2020-05-11', '2024-04-01'],
})
halving_dates['date'] = pd.to_datetime(halving_dates['date'])

In [ ]:
predicted_d_ln_price_df = pd.read_csv('../bitcoinforum/7_econometrics/arima_predictions_full_data.csv') # date	predicted_d_ln_price
predicted_d_ln_price_df['date'] = pd.to_datetime(predicted_d_ln_price_df['date'])
ln_price = np.log(price)
predicted_price = predicted_d_ln_price_df.copy()

# go through each (date, predicted_d_ln_price) pair and obtain the predicted price of the next quarter
for i, row in predicted_price.iterrows():
    date = row['date']
    predicted_d_ln_price = row['predicted_d_ln_price']
    # calculate the predicted price of the next quarter
    predicted_price.at[i, 'predicted_ln_price'] = ln_price.loc[date] + predicted_d_ln_price
    predicted_price.at[i, 'ln_price'] = ln_price.loc[date]

# shift the predicted price by one index
predicted_price['predicted_ln_price'] = predicted_price['predicted_ln_price'].shift(1)
predicted_price = predicted_price.dropna()
predicted_price['predicted_price'] = np.exp(predicted_price['predicted_ln_price'])
predicted_price['price'] = np.exp(predicted_price['ln_price'])
predicted_price.head(2)

In [ ]:
if THESIS:
    electricity_costs = np.array([0.03, 0.07]) * PUE / (1000*3600)
    aux_costs = np.array([1, 2.0])
else:
    electricity_costs = np.array([0.03, 0.15]) / (1000*3600)
    aux_costs = np.array([1, 2.0])
colors = ['green', 'red']
names = ['m_min(t)', 'm_max(t)']

# func = np.log10
func = lambda x: x

# Calculate cost per coin for each combination
cost_per_coin = [func(joules_per_coin * e * a) for (e, a) in zip(electricity_costs, aux_costs)]
cost_per_coin = [x.rename(columns={'joules_per_coin': names[i]}) for i, x in enumerate(cost_per_coin)]
cost_per_coin = pd.concat(cost_per_coin, axis=1)
cost_per_coin = cost_per_coin.iloc[::-1]

fig, ax = plt.subplots(figsize=(12, 6))  # Increase the width of the plot


# cost_per_coin = cost_per_coin.resample('Q').mean()
# log_price = log_price.resample('Q').mean()
# price = price.resample('Q').mean()
# predicted_price = predicted_price.set_index('date').resample('Q').mean().reset_index()


# Plot each cost per coin with the specified colors and labels using LaTeX for subscript
for i, color in enumerate(colors):
    ax.plot(cost_per_coin.index, cost_per_coin[names[i]], linestyle='--', color=color, label=f'{names[i]}')


# Plot log_price with a thick black line
ax.plot(log_price.index, 10**log_price, alpha=1, color='black', linewidth=2, label='price')

# Add vertical lines for halving events
halving_dates = pd.to_datetime(['2012-11-28', '2016-07-09', '2020-05-11', '2024-04-01'])
for date in halving_dates[:-1]:
    ax.axvline(x=date, color='blue', linestyle='--', alpha=0.7, label='halving events' if date == halving_dates[0] else "")

# Set x-ticks to every year and format them to include "01.01."
years = pd.date_range(start=Timestamp('2010-12-31 00:00:00'), end=cost_per_coin.index.max(), freq='YS')
ax.set_xticks(years)

# ax.set_xticklabels([f"01.01.{year.year}" for year in years], rotation=30)
ax.set_xticklabels([f"{year.year}" for year in years], rotation=30)

# Set y-axis label and format y-ticks to display as 10^x
ax.set_ylabel('Value (USD)', rotation='vertical')
# ax.set_yscale('log')
# ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'$10^{{{int(x)}}}$'))
ax.set_yscale('log')
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'$10^{{{int(np.log10(x))}}}$'))


# Interpolate the initial price for the start_date
start_date = pd.Timestamp('2013-12-31')
if start_date not in price.index:
    initial_price = price.reindex(price.index.union([start_date])).interpolate(method='time').loc[start_date]
else:
    initial_price = price.loc[start_date]

# Add Koomey's post-2000 law
# end_date = pd.Timestamp('2030-12-31')
end_date = pd.Timestamp('2023-10-31')
koomey_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Koomey's law: Doubling efficiency every 2.6 years
koomey_values = (initial_price + 1000) * (2 ** ((koomey_dates - start_date) / pd.Timedelta(days=2.6 * 365.25)))

# Plot Koomey's law
plt.plot(koomey_dates, koomey_values, label="Koomey's post-2000 Law", color='orange')

# # Plot the predicted price
plt.plot(predicted_price['date'], predicted_price['predicted_price'], label='Predicted Price with ARIMA', linestyle=':', color='black', linewidth=3)

# add a grid
ax.grid(True)
# ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=len(colors) + 2, fancybox=True, shadow=True, fontsize=20)
ax.legend(fontsize=18)

plt.rcParams.update({'font.size': 18})



plt.savefig('pdfs/pricepredictions.pdf', format='pdf', bbox_inches='tight')
plt.show()
plt.rcParams.update({'font.size': fs})

In [ ]:
colors = ['blue', 'blue']
if THESIS:
    electricity_costs = np.array([0.03, 0.07]) * PUE / (1000*3600)
    aux_costs = np.array([1, 2.0])
    names = ['m_min(t)', 'm_max(t)']
    names2 = ['m_min(t)', 'm_max(t)']
else:
    electricity_costs = np.array([0.03, 0.05, 0.07]) / (1000*3600)
    aux_costs = np.array([1, 1.1, 1.2])
    names = ['m_min(t)', 'm_middle(t)', 'm_max(t)']
    names2 = ['C = 0.03 USD/kWh; PUE = 1.0', 'C = 0.05 USD/kWh; PUE = 1.1', 'C = 0.07 USD/kWh; PUE = 1.2']
    colors.append('blue')

In [ ]:
# Left plot stuff:
# func = np.log10
func = lambda x: x

# Calculate cost per coin for each combination
cost_per_coin = [func(joules_per_coin * e * a) for (e, a) in zip(electricity_costs, aux_costs)]
cost_per_coin = [x.rename(columns={'joules_per_coin': names[i]}) for i, x in enumerate(cost_per_coin)]
cost_per_coin = pd.concat(cost_per_coin, axis=1)
cost_per_coin = cost_per_coin.iloc[::-1]




# Right plot stuff:
# Convert index to datetime
joules_per_second.index = pd.to_datetime(joules_per_second.index)

# Resample to daily frequency and interpolate
daily_joules_per_second = joules_per_second.resample('D').interpolate(limit_direction='both')

# Convert joules per second to TWH
twh_per_day = daily_joules_per_second * 24 * 1e-12

# Apply PUE
twh_per_day = twh_per_day * PUE

# Calculate cumulative TWH
cumulative_twh = twh_per_day.cumsum()





In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
intensity_file = "carbonintensity.html"
# read the html file using the first row as header
dfs = pd.read_html(intensity_file, header=0)

# Access the second table (index 1)
df = dfs[0]
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Display the DataFrame
df.head(1) # Date	China (% hashrate)	China emission intensity (in gCO2e/kwh)	Rest (% hashrate)	Rest of the world emission intensity (in gCO2e/kwh)	World emission intensity (in gCO2e/kwh)

In [ ]:
average_values = df.groupby(df.index.year).mean()
# for each year, get the average intensity (weighted by hashrate)
average_values['intensity'] = (average_values['China emission intensity (in gCO2e/kwh)'] * average_values['China (% hashrate)'] + average_values['Rest of the world emission intensity (in gCO2e/kwh)'] * average_values['Rest (% hashrate)']) / 100
# the df ends at 2019. forward fill up to 2023
intensity_df = average_values.reindex(range(2011, 2024)).ffill()
intensity_df["intensity"] = intensity_df["intensity"]*1e-3
intensity_df["intensity"]
intensity_df2 = pd.DataFrame(intensity_df["intensity"])
intensity_df2["year"] = intensity_df2.index
intensity_df2.to_csv("carboncomparison/intensity.csv", index=False)
intensity_df2

In [ ]:
# Create table where for each Jan 1st starting with 2012, we have the cumulative and daily TWH
electricity_table = pd.DataFrame(index=pd.date_range(start='2011-01-01', end='2023-01-01', freq='YS'))
electricity_table['daily_twh'] = twh_per_day.resample('YS').last()
electricity_table['yearly_twh'] = twh_per_day.resample('YS').sum()
electricity_table['cumulative_twh'] = cumulative_twh.resample('YS').last()

# # Calculate carbon footprint (0.5 gCO2/Wh = 0.5e-6 TCO2/Wh)
# intensity = 0.5e-6  # TCO2/Wh
# electricity_table['daily_carbon'] = electricity_table['daily_twh'] * 1e12 * intensity
# electricity_table['yearly_carbon'] = electricity_table['yearly_twh'] * 1e12 * intensity
# electricity_table['cumulative_carbon'] = electricity_table['cumulative_twh'] * 1e12 * intensity

# Calculate carbon footprint with intensity_df
# Ensure intensity_df has the same index as electricity_table
intensity_df.index = pd.to_datetime(intensity_df.index.astype(str) + '-01-01')
# Merge electricity_table with intensity_df
merged_table = electricity_table.merge(intensity_df[['intensity']], left_index=True, right_index=True, how='left')

# Calculate carbon footprint
multi = 1e6
merged_table['daily_carbon'] = merged_table['daily_twh'] * merged_table['intensity'] * multi
merged_table['yearly_carbon'] = merged_table['yearly_twh'] * merged_table['intensity'] * multi
merged_table['cumulative_carbon'] = merged_table['cumulative_twh'] * merged_table['intensity'] * multi


electricity_table = merged_table.drop(columns=['intensity'])
electricity_table["intensity"] = merged_table["intensity"]


electricity_table = electricity_table.T
electricity_table.columns = electricity_table.columns.year  # Use only the year for column names
electricity_table.index = ['Daily TWH', 'Yearly TWH', 'Cumulative TWH', 'Daily Carbon (TCO2)', 'Yearly Carbon (TCO2)', 'Cumulative Carbon (TCO2)', 'Intensity (gCO2/Wh)']

# Add a row to separate electricity consumption from carbon footprint
separator = pd.DataFrame([[''] * len(electricity_table.columns)], index=[''], columns=electricity_table.columns)
electricity_table = pd.concat([electricity_table.iloc[:3], separator, electricity_table.iloc[3:]])


electricity_table = electricity_table.T



# Print the table in LaTeX format
print(electricity_table.to_latex(float_format="%.1e", index=True))

In [ ]:
electricity_table.to_csv("../excel_sheet/csv/Electricity consumption and carbon footprint.csv")

In [ ]:
yearlycarbon = merged_table[['yearly_carbon'] ]
yearlycarbon["year"] = yearlycarbon.index.year
yearlycarbon.rename(columns={"yearly_carbon": "ours"}, inplace=True)
yearlycarbon.to_csv('carboncomparison/01_ours.csv', index=False)

In [ ]:
yearlycarbon

In [ ]:
electricity_table

In [ ]:

# Define the first plotting function
def plot_cost_per_coin(ax, cost_per_coin, log_price, names, names2, colors):
    # Add more gridlines
    ax.grid(which='major', color='#DDDDDD', linewidth=0.8)
    ax.grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.8)
    ax.minorticks_on()

    start_date = cost_per_coin.index[0] - pd.DateOffset(months=0)
    end_date = cost_per_coin.index[-1] + pd.DateOffset(months=0)
    ax.set_xlim(start_date, end_date)

    ax.plot(log_price.index, 10**log_price, alpha=1, color='black', linewidth=0.8, label="Price of Bitcoin")

    # Plot each cost per coin with the specified colors and labels using LaTeX for subscript
    # for i, color in enumerate(colors):
    for i in range(3)[::-1]:
        # print(i)
        # c = color
        c = fancycolors[i]
        ax.plot(cost_per_coin.index, cost_per_coin[names[i]], linestyle='-', linewidth=0.8, color=c, label=f'{names2[i]}')


    

    # Fill the area between m_min(t) and m_max(t) (old color was "lightblue")
    ax.fill_between(cost_per_coin.index, cost_per_coin['m_min(t)'], cost_per_coin['m_max(t)'], color="lightblue", alpha=0.5)

    # Set x-ticks to every year and format them to include "01.01."
    years = pd.date_range(start=pd.Timestamp('2010-12-31 00:00:00'), end=('2024-01-01 00:00:00'), freq='YS')
    ax.set_xticks(years)
    ax.set_xticklabels([f"{year.year}" for year in years], rotation=30)

    # Set y-axis label and format y-ticks to display as 10^x
    ax.set_ylabel('Value (USD)', rotation='vertical')

    ax.set_yscale('log')
    # ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'$10^{{{int(np.log10(x))}}}$'))
    ax.set_yscale('log')



    # Add legend with the specified changes
    handles, labels = ax.get_legend_handles_labels()
    labels = labels
    ax.legend(handles, labels, loc='lower right', fancybox=True, shadow=True, fontsize=fs)






plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams.update({'font.size': fs})
plt.rcParams.update({'figure.dpi': 150})
plt.rcParams['figure.facecolor'] = 'white'


fig, ax = plt.subplots(figsize=(12, 6))
plot_cost_per_coin(ax, cost_per_coin, log_price, names, names2, colors)
plt.savefig('pdfs/enveloppe_wide.pdf', format='pdf', bbox_inches='tight')
plt.show()
fig, ax = plt.subplots(figsize=(6, 6))
plot_cost_per_coin(ax, cost_per_coin, log_price, names, names2, colors)
plt.savefig('pdfs/enveloppe_thin.pdf', format='pdf', bbox_inches='tight')
plt.show()



In [ ]:
twh_per_day # series: date

In [ ]:
intensity_df2 #	Date intensity	year	

In [ ]:
mapping = {}
for row in intensity_df2.itertuples():
    year = row.year
    intensity = row.intensity
    mapping[year] = intensity

In [ ]:
co2_per_day = pd.DataFrame(twh_per_day)# * 1e6
for (i,row) in enumerate(co2_per_day.iterrows()):
    year = row[0].year
    co2_per_day.iloc[i] *= mapping[year]

In [ ]:
co2_per_day = co2_per_day[0]
co2_per_day

In [ ]:
cumulative_co2 = co2_per_day.cumsum()

In [ ]:
def plot_twh(ax, daily_joules_per_second, twh_per_day, cumulative_twh, co2):
    # Get average twh_per_day on a quarterly basis for smoothing
    quarterly_twh_per_day = twh_per_day.resample('Q').mean()

    # Convert it back to daily
    quarterly_twh_per_day = quarterly_twh_per_day.resample('D').interpolate()
    # Make it end at the same day as the last day of the daily data
    quarterly_twh_per_day = quarterly_twh_per_day[:twh_per_day.index[-1]]
    # Trim cumulative_twh to start at the same day as quarterly_twh_per_day
    start_date = quarterly_twh_per_day.index[0]
    cumulative_twh = cumulative_twh[start_date:]

    # Plot daily TWH (linestyle: dots on solid line, with markers only once per month)
    # ax.plot(quarterly_twh_per_day.index, quarterly_twh_per_day, label='Daily', linestyle='-', marker='o', markevery=90, color=fancyblue, linewidth=0.8)

    # Calculate yearly consumption
    yearly_twh_per_day = twh_per_day.resample('Y').sum()
    # Delay it by -1 years
    yearly_twh_per_day.index = yearly_twh_per_day.index - pd.DateOffset(years=1)

    # Plot yearly consumption as black triangles and connect them with a black line
    # ax.plot(yearly_twh_per_day.index, yearly_twh_per_day, label='Yearly', linestyle='-', marker='^', color='black', linewidth=0.8)
    # Plot yearly consumption as semi-transparent light gray bars
    yearly_plot = ax.bar(yearly_twh_per_day.index, yearly_twh_per_day, 
                         color='black', alpha=0.8, width=180, zorder=3)

    # Plot cumulative TWH 
    cumulative_plot = ax.plot(
        # cumulative_twh.index, cumulative_twh, 
        # cumulative_twh.resample('Y').first().index, cumulative_twh.resample('Y').last(),
        yearly_twh_per_day.cumsum().index, yearly_twh_per_day.cumsum(),
                              linestyle='-', marker='x', markevery=1, 
                              color='red', linewidth=0.8, zorder=4)[0]





    # Set log scale
    ax.set_yscale('log')

    # Set x-axis to show every year but only the year
    years = pd.date_range(start=pd.Timestamp('2010-12-31 00:00:00'), end=daily_joules_per_second.index.max(), freq='YS')
    ax.set_xticks(years)
    ax.set_xticklabels([year.year for year in years], rotation=30)

    # Add more gridlines
    ax.grid(which='major', color='#DDDDDD', linewidth=0.8, zorder=0)
    ax.grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.8, zorder=0)
    ax.minorticks_on()
    ax.xaxis.set_minor_locator(plt.NullLocator())

    # Add labels
    # ax.set_xlabel('Date')

    if co2:
        ax.set_ylabel('Carbon footprint ($\mathrm{MtCO_2}$)')
    else:
        ax.set_ylabel('Electricity consumption (TWh)')

    # Add legend
    handles = [yearly_plot, cumulative_plot]
    labels = ['Yearly', 'Cumulative']
    ax.legend(handles, labels, fontsize=fs)

plt.rcParams['mathtext.fontset'] = 'stix'


fig, ax = plt.subplots(figsize=(6, 6))
plot_twh(ax, daily_joules_per_second, twh_per_day, cumulative_twh, False)
plt.savefig('pdfs/climate_thin.pdf', format='pdf', bbox_inches='tight')
plt.show()
fig, ax = plt.subplots(figsize=(12, 6))
plot_twh(ax, daily_joules_per_second, twh_per_day, cumulative_twh, False)
plt.savefig('pdfs/climate_wide.pdf', format='pdf', bbox_inches='tight')
plt.show()


fig, ax = plt.subplots(figsize=(12, 6))
plot_twh(ax, daily_joules_per_second, co2_per_day, cumulative_co2, True)
plt.savefig('pdfs/climate_wide_co2.pdf', format='pdf', bbox_inches='tight')
plt.show()
fig, ax = plt.subplots(figsize=(6, 6))
plot_twh(ax, daily_joules_per_second, co2_per_day, cumulative_co2, True)
plt.savefig('pdfs/climate_thin_co2.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
if THESIS:
    pass
else:
    names = ['m_min(t)', 'm_middle(t)', 'm_max(t)']
    # names2 = ['C = 0.03 USD/kWh; PUE = 1.0', 'C = 0.05 USD/kWh; PUE = 1.1', 'C = 0.07 USD/kWh; PUE = 1.2']


# Calculate cost per coin for each combination and log-transform them
cost_per_coin = [np.log10(joules_per_coin * e * a) for (e, a) in zip(electricity_costs, aux_costs)]
cost_per_coin = [x.rename(columns={'joules_per_coin': names[i]}) for i, x in enumerate(cost_per_coin)]
cost_per_coin = pd.concat(cost_per_coin, axis=1)
cost_per_coin = cost_per_coin.iloc[::-1]

# Calculate the price minus each of the cost lines
price_minus_cost = pd.DataFrame(index=cost_per_coin.index)
for name in names:
    price_minus_cost[name] = log_price - cost_per_coin[name]


def plot_profit_per_dollar(ax):
    global colors, names, names2
    ax.minorticks_on()

    start_date = cost_per_coin.index[0] - pd.DateOffset(months=0)
    end_date = cost_per_coin.index[-1] + pd.DateOffset(months=0)
    ax.set_xlim(start_date, end_date)

    # Plot each price minus cost per coin with the specified colors and labels using LaTeX for subscript
    for i, color in enumerate(colors):
        lab = names2[i]
        c = fancycolors[i]
        ax.semilogy(price_minus_cost.index, 10**price_minus_cost[names[i]], linestyle='-', linewidth=0.8, color=c, label=lab)

    # Fill the area between price - m_min(t) and price - m_max(t) with light blue shade
    ax.fill_between(price_minus_cost.index, 10**price_minus_cost['m_min(t)'], 10**price_minus_cost['m_max(t)'], color='lightblue', alpha=0.5)

    # Set x-ticks to every year and format them to include "01.01."
    years = pd.date_range(start=pd.Timestamp('2010-12-31 00:00:00'), end=('2024-01-01 00:00:00'), freq='YS')
    ax.set_xticks(years)
    ax.set_xticklabels([f"{year.year}" for year in years], rotation=30)

    # Set y-axis label
    ax.set_ylabel('Price of Bitcoin to mining costs ratio', rotation='vertical')

    # Set font to Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'

    # Make the font larger
    fs = 14
    plt.rcParams.update({'font.size': fs})

    # Increase DPI
    plt.rcParams.update({'figure.dpi': 100})

    # Rotate x-ticks
    plt.xticks(rotation=30)

    # Add a grid for log scale
    ax.grid(which='both', color='#DDDDDD', linestyle=':', linewidth=0.8)
    ax.set_yscale('log')

    # Set y-axis ticks
    ax.yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=15))
    ax.yaxis.set_minor_locator(ticker.LogLocator(base=10, subs=np.arange(2, 10) * 0.1, numticks=15))

    # Custom formatter for y-axis labels
    def custom_formatter(y, pos):
        if y == 1:
            return '1'
        elif y == 10:
            return '10'
        elif y >= 100:
            return f'$10^{int(np.log10(y))}$'
        else:
            return f'$10^{np.log10(y):.1f}$'

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(custom_formatter))

    ax.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=1, fancybox=True, shadow=True)

fig, ax = plt.subplots(figsize=(12, 6))
plot_profit_per_dollar(ax)
plt.savefig('pdfs/profit_wide.pdf', format='pdf', bbox_inches='tight')
plt.show()
fig, ax = plt.subplots(figsize=(6, 6))
plot_profit_per_dollar(ax)
plt.savefig('pdfs/profit_thin.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
# for each column in cost_per_coin, take 10^x
for col in cost_per_coin.columns:
    cost_per_coin[col] = 10**cost_per_coin[col]

In [ ]:
# Calculate daily gains for each mining cost
daily_gains = pd.DataFrame(index=cost_per_coin.index)
# for i in range(5):
#     daily_gains[f'm_{i}(t)'] = (10**log_price - cost_per_coin[f'm_{i}(t)']) / cost_per_coin[f'm_{i}(t)']
for name in names[::-1]:
    daily_gains[f"{name}'s profits"] = (10**log_price - cost_per_coin[name]) / cost_per_coin[name]
# daily_gains["Profit"] = (10**log_price - cost_per_coin[f'm_min(t)']) / cost_per_coin[f'm_min(t)']

# Resample to yearly and calculate the average daily gains
yearly_gains = daily_gains.resample('Y').mean()

# Convert the gains to percentage increase
yearly_gains_percentage = yearly_gains * 100

# Format the gains as strings with percentage increase
yearly_gains_str = yearly_gains_percentage.applymap(lambda x: f'{x:+.0f}%')

# Transpose the DataFrame to have years as columns and mining costs as rows
profits_df = yearly_gains_str.T

# Display the DataFrame
profits_df

same table again with fancy colors:

In [ ]:
# Calculate the log10 of the absolute values of the profits
log_abs_profits = profits_df.applymap(lambda x: np.log10(abs(float(x.strip('%').strip('+').strip('\\')) / 100)))
sign_profits = profits_df.applymap(lambda x: 1 if '+' in x else -1)

eps = 1e-6
# Normalize the log profits for positive and negative values separately
max_positive = log_abs_profits[sign_profits > 0].max().max()
min_positive = log_abs_profits[sign_profits > 0].min().min() + eps
max_negative = log_abs_profits[sign_profits < 0].max().max()
min_negative = log_abs_profits[sign_profits < 0].min().min() + eps

def get_color(value, sign):
    try: 
        if sign > 0:
            normalized_value = (value - min_positive) / (max_positive - min_positive)
            red = int((1 - normalized_value*0.3) * 255)
            green = 255
        else:
            normalized_value = (value - min_negative) / (max_negative - min_negative)
            red = 255
            green = int((1 - normalized_value*0.3) * 255)
        return f'\\cellcolor[RGB]{{{red},{green},50}}'
    except:
        return f'\\cellcolor[RGB]{{0,0,0}}'

# Start the LaTeX table
latex_code = "\\documentclass{article}\n\\usepackage{colortbl}\n\\usepackage{booktabs}\n\\usepackage{graphicx}\n\\usepackage{array}\n\\begin{document}\n"
latex_code += "\\newcolumntype{C}[1]{>{\\centering\\arraybackslash}p{#1}}\n"
latex_code += "\\begin{table}[ht]\n\\centering\n\\resizebox{\\textwidth}{!}{%\n\\begin{tabular}{l" + "C{1.5cm}" * len(profits_df.columns) + "}\n"
latex_code += "\\toprule\n"

# Add the header row
header_row = " & " + " & ".join(profits_df.columns.strftime('%Y')) + " \\\\\n"
latex_code += header_row
latex_code += "\\midrule\n"

# Add the data rows with background colors
for index, row in profits_df.iterrows():
    row_data = " & ".join([f"{get_color(log_abs_profits.at[index, col], sign_profits.at[index, col])}" + val.replace('%', '\\%') for col, val in row.items()])
    latex_code += f"{index} & {row_data} \\\\\n"

# End the LaTeX table
latex_code += "\\bottomrule\n"
latex_code += "\\end{tabular}}\n\\caption{Yearly Profits as Percentage Increase}\n\\label{tab:profits}\n\\end{table}\n"
latex_code += "\\end{document}"

latex_code = latex_code.replace("m_max(t)", "$\\mathrm{m_{min}(t)}$").replace("m_middle(t)", "$\\mathrm{m_{middle}(t)}$").replace("m_min(t)", "$\\mathrm{m_{max}(t)}$")

# Print the LaTeX code
print(latex_code)

In [ ]:
price = 10**log_price

price = pd.DataFrame(price)
price = price.rename(columns={0: 'price'})
# price["c_mining"] = cost_per_coin["m_2(t)"]
# price

In [ ]:
# cost_per_coin # date m_min(t)	m_max(t)

In [ ]:

# Calculate log10 of price and c_mining
price['log_price'] = np.log10(price['price'])

# Plot the spread over time
plt.figure(figsize=(12, 6))

i = 0
for thing in [np.log10(cost_per_coin['m_min(t)']), np.log10(cost_per_coin['m_max(t)'])]:
    # Perform linear regression to find alpha and beta
    X = thing
    # print(X)
    y = price['log_price']
    X = sm.add_constant(X)  # Adds a constant term to the predictor

    # at some point this broke
    model = sm.OLS(y, X).fit()
    alpha = model.params['const']
    beta = model.params[thing.name]
    print(f"Alpha: {alpha}")
    print(f"Beta: {beta}")
    # beta = 1; alpha = 0



    # Calculate the spread
    # spread = price['log_price'] - beta * thing - alpha
    spread = price['log_price'] - thing
    plt.plot(price.index, spread, label="Spread for $" + names[i].replace("m_","m_{").replace("(","}(") + "$", color=colors[i])
    i += 1

spread = price['log_price'] - beta * thing - alpha
plt.xlabel('Date')
plt.ylabel('Spread')
plt.title('Spread over Time')
plt.legend(fontsize=17)


# set_xticklabels
years = pd.date_range(start=Timestamp('2010-12-31 00:00:00'), end=price.index.max(), freq='YS')
plt.xticks(years, [f"01.01.{year.year}" for year in years], rotation=30)
plt.xticks(rotation=30)
plt.grid(True)

# Add a thick line at y=0
plt.axhline(y=0, color='black', linewidth=2)
plt.rcParams.update({'font.size': 17})
plt.savefig('pdfs/spread.pdf', format='pdf', bbox_inches='tight')
plt.show()
plt.rcParams.update({'font.size': fs})
spread_full = spread
spread_before_turning_point = spread.loc[:'2013-07-01']
spread_after_turning_point = spread.loc['2013-07-01':]


In [ ]:
# # Perform Augmented Dickey-Fuller test
# adf_result = adfuller(price['spread'])
# print('ADF Statistic:', adf_result[0])
# print('p-value:', adf_result[1])
# for key, value in adf_result[4].items():
#     print('Critical Values:')
#     print(f'   {key}, {value}')

# # Interpretation of the ADF test
# if adf_result[1] < 0.05:
#     print("The spread is stationary (reject the null hypothesis of a unit root).")
# else:
#     print("The spread is not stationary (fail to reject the null hypothesis of a unit root).")


from statsmodels.stats.diagnostic import acorr_ljungbox

# Perform Ljung-Box test
ljung_box_result = acorr_ljungbox(spread_full, lags=[10], return_df=True)
print(ljung_box_result)

# Interpretation of the Ljung-Box test
if ljung_box_result['lb_pvalue'].values[0] < 0.05:
    print("The spread is not white noise (reject the null hypothesis of no autocorrelation).")
else:
    print("The spread is white noise (fail to reject the null hypothesis of no autocorrelation).")


# from statsmodels.tsa.stattools import variance_ratio

# # Perform Variance Ratio test
# vr_result = variance_ratio(price['spread'])
# print('Variance Ratio:', vr_result)

# # Interpretation of the Variance Ratio test
# if vr_result < 1:
#     print("The spread is likely to be mean-reverting (not a random walk).")
# else:
#     print("The spread is likely to be a random walk (noise).")


# from scipy.signal import periodogram

# # Perform Spectral Analysis
# frequencies, spectrum = periodogram(price['spread'])

# plt.figure(figsize=(12, 6))
# plt.plot(frequencies, spectrum)
# plt.xlabel('Frequency')
# plt.ylabel('Spectral Density')
# plt.title('Spectral Density of the Spread')
# plt.show()

# Interpretation: A flat spectral density indicates white noise.

In [ ]:
# Calculate archaicity profits
df = pd.read_csv('../bitcoinforum/6_merging/monthly_stuff.csv')
df['date'] = pd.to_datetime(df['date'])
seconds_in_block = 600
df = df[['date', 'hashrate', 'price', 'max_efficiency', 'coins_per_block']]
df = df.iloc[::-1]
df['max_efficiency'] = df['max_efficiency'].ffill()
df['max_efficiency'] = df['max_efficiency'] * 1e12
df.index = df['date']
df = df.drop(columns=['date'])
df

In [ ]:
# Constants
electricity_cost_per_joule = 0.10 / (1000 * 3600)  # USD per joule
seconds_in_block = 600

# Function to calculate mining cost
def calculate_mining_cost(hashrate, max_efficiency, coins_per_block):
    joules_per_hash = 1 / max_efficiency
    joules_per_block = hashrate * seconds_in_block * joules_per_hash
    cost_per_block = joules_per_block * electricity_cost_per_joule
    return cost_per_block / coins_per_block

# Initialize a DataFrame to store the results
archaicity_df = pd.DataFrame(index=df.index, columns=['archaicity_months'])

# Iterate through each month
for current_month in df.index:
    max_efficiency = df.loc[current_month, 'max_efficiency']
    archaicity_months = 0
    
    # Iterate through the next months
    for future_month in df.loc[current_month:].index:
        future_hashrate = df.loc[future_month, 'hashrate']
        future_price = df.loc[future_month, 'price']
        future_coins_per_block = df.loc[future_month, 'coins_per_block']
        
        # Calculate mining cost
        mining_cost = calculate_mining_cost(future_hashrate, max_efficiency, future_coins_per_block)
        
        # Check if mining is still profitable
        if mining_cost < future_price:
            archaicity_months += 1
        else:
            break
    
    # Store the result
    archaicity_df.loc[current_month, 'archaicity_months'] = archaicity_months

# Resample to yearly and take the mean
yearly_archaicity = archaicity_df.resample('Y').mean()

print(yearly_archaicity)

In [ ]:
# quarterly_archaicity = archaicity_df.resample('Q').mean()
# quarterly_archaicity.plot()

# monthly_archaicity = archaicity_df.resample('M').mean()
# monthly_archaicity.plot()

# year2_archaicity = archaicity_df.resample('2Y').mean()
# year2_archaicity.plot()

In [ ]:
# latex_table = yearly_archaicity.to_latex(index=True, header=True, caption="Yearly Archaicity of Bitcoin Mining Hardware", label="tab:yearly_archaicity", float_format="%.2f")
# print(latex_table)

In [ ]:
transposed_yearly_archaicity = yearly_archaicity.T
transposed_yearly_archaicity = transposed_yearly_archaicity*30.44
# Export to LaTeX
latex_table = transposed_yearly_archaicity.to_latex(index=False,float_format="%.2f")
print(latex_table)

In [ ]:
# Calculate archaicity
df = pd.read_csv('../bitcoinforum/6_merging/monthly_stuff.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[['date', 'efficiency', 'max_efficiency']]
df = df.iloc[::-1]
df['max_efficiency'] = df['max_efficiency'].ffill()
df['max_efficiency'] = df['max_efficiency'] * 1e12
df.index = df['date']
df = df.drop(columns=['date'])
df

In [ ]:
# Initialize a DataFrame to store the results
efficiency_df = pd.DataFrame(index=df.index, columns=['months_until_surpassed'])

# Iterate through each month
for current_month in df.index:
    max_efficiency = df.loc[current_month, 'max_efficiency']
    months_until_surpassed = 0
    
    # Iterate through the next months
    for future_month in df.loc[current_month:].index:
        future_efficiency = df.loc[future_month, 'efficiency']
        
        # Check if future efficiency surpasses current month's max efficiency
        if future_efficiency > max_efficiency:
            break
        else:
            months_until_surpassed += 1
    
    # Store the result
    efficiency_df.loc[current_month, 'months_until_surpassed'] = months_until_surpassed

# Resample to yearly and take the mean
yearly_archaicity = efficiency_df.resample('Y').mean()

print(yearly_archaicity)

In [ ]:
# archaicity_df = efficiency_df
# quarterly_archaicity = archaicity_df.resample('Q').mean()
# quarterly_archaicity.plot()

# monthly_archaicity = archaicity_df.resample('M').mean()
# monthly_archaicity.plot()

# year2_archaicity = archaicity_df.resample('2Y').mean()
# year2_archaicity.plot()

In [ ]:
transposed_yearly_archaicity = yearly_archaicity.T
transposed_yearly_archaicity = transposed_yearly_archaicity*30.44
transposed_yearly_archaicity

In [ ]:
# Export to LaTeX
latex_table = transposed_yearly_archaicity.to_latex(index=False,float_format="%.2f")
print(latex_table)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(yearly_archaicity.index, yearly_archaicity*30.44, marker='o', linestyle='-')
plt.title('Average Days Until Efficiency Surpassed (Yearly)')
plt.xlabel('Year')
plt.ylabel('Average Days Until Surpassed')
plt.grid(True)
plt.show()

In [ ]:
df = pd.read_csv('../bitcoinforum/6_merging/monthly_stuff.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[['date', 'efficiency', 'max_efficiency']]
df = df.iloc[::-1]
df['max_efficiency'] = df['max_efficiency'].ffill()
df['max_efficiency'] = df['max_efficiency'] * 1e12
df.index = df['date']
df = df.drop(columns=['date'])
df

In [ ]:
pmaxv2 = pd.read_csv('../hardwarelist/pmaxv2.csv') # date	max_efficiency
pmaxv2['date'] = pd.to_datetime(pmaxv2['date'])
pmaxv2 = pmaxv2.set_index('date')
pmaxv2

In [ ]:
# for each date in df, get the max_efficiency from pmaxv2
df['max_efficiency'] = df.index.map(pmaxv2['max_efficiency'])
# make df quarterly
df = df.resample('Q').mean()
df.tail(10)

In [ ]:
# set dpi to 100
plt.rcParams.update({'figure.dpi': 100})

df.plot()

In [ ]:
# plot log10(efficiency) - log10(max_efficiency)
plt.figure(figsize=(10, 6))
plt.plot(df.index, np.log10(df['efficiency']) - np.log10(df['max_efficiency']))
plt.xlabel('Date')
plt.ylabel('log10(efficiency) - log10(max_efficiency)')
plt.title('Difference between Efficiency and Max Efficiency')
plt.grid(True)
plt.show()


# part 2: data plot, yearly hardware count, efficiencycomparison

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from datetime import timedelta
warnings.filterwarnings('ignore')
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import seaborn as sns
from scipy.stats import gaussian_kde
from datetime import datetime, timedelta

plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
# Define the variable name for the unit power efficiency column
unit_power_efficiency = 'TH/J'

data = pd.read_csv('../bitcoinforum/5_processing_extracted_data/hardware_instances_with_efficiency.csv') # date,row_index,hardware_name,TH/J

# Convert the 'date' column to a datetime format
data['date'] = pd.to_datetime(data['date'])

# keep only data from 2011 onwards
data = data[data['date'] >= '2011-01-05']
data

In [ ]:
# Step 1: Average the data points on a quarterly basis
# Ensure that 'TH/J' is a numeric type
data['TH/J'] = pd.to_numeric(data['TH/J'], errors='coerce')

# Create a 'quarter' column by offsetting the 'date' by 15 days and then using to_period
data['quarter'] = (data['date'] - timedelta(days=45)).dt.to_period('Q')
# data['quarter'] = (data['date'] - timedelta(days=0)).dt.to_period('Q')
data

In [ ]:
# Group by 'quarter' and calculate the mean of 'TH/J'
quarterly_data = data.groupby('quarter')['TH/J'].mean().reset_index()

# Convert 'quarter' back to datetime (last day of the quarter)
quarterly_data['quarter'] = quarterly_data['quarter'].dt.to_timestamp(freq='Q')
# # Convert 'quarter' back to datetime (first day of the quarter)
# quarterly_data['quarter'] = quarterly_data['quarter'].dt.to_timestamp()
quarterly_data.head(3)

In [ ]:
# add a 2011-01-01 row to the quarterly_data and copy the first row to it
# create timestamp for 2011-01-01
ts = pd.Timestamp('2011-01-01')
quarterly_data.loc[-1] = [ts, quarterly_data['TH/J'][0]]
quarterly_data.index = quarterly_data.index + 1
quarterly_data = quarterly_data.sort_index()
quarterly_data

In [ ]:
# obtain monthly data
data['month'] = data['date'].dt.to_period('M')
monthly_data = data.groupby('month')['TH/J'].mean().reset_index()
monthly_data['month'] = monthly_data['month'].dt.to_timestamp()

In [ ]:
dpi = 110


# unit = "TH/J"
unit = "H/J"
# func = "ln"
func = "log10"
def process(x,apply_multiplier=False):
    multi = 1e12 if unit == "H/J" else 1
    if apply_multiplier == False:
        multi = 1
    if func == "ln":
        return np.log(x*multi)
    if func == "log10":
        return np.log10(x*multi)

In [ ]:
process(data[unit_power_efficiency])

In [ ]:
gray_scatter_x = data['date'] # looks like 2011-02-19 09:38:19
gray_scatter_y = process(data[unit_power_efficiency],apply_multiplier=True) # looks like 6.302179

red_scatter_x = quarterly_data['quarter'] # looks like 2011-01-01
red_scatter_y = process(quarterly_data['TH/J'],apply_multiplier=True) # looks like 6.217607 (average of that time period)

lime_scatter_x = monthly_data['month'] # looks like 2011-01-01
lime_scatter_y = process(monthly_data['TH/J'],apply_multiplier=True) # looks like 6.217607 (average of that time period)

# Load the data for the maximum efficiency for all dates
max_efficiency_table = pd.read_csv('../hardwarelist/pmaxv3.csv')
max_efficiency_table['date'] = pd.to_datetime(max_efficiency_table['date'])
max_plot_x = max_efficiency_table['date']
max_plot_y = process(max_efficiency_table['max_efficiency']*1e-12,apply_multiplier=True)

In [ ]:
monthly_price_joules = pd.read_csv('../bitcoinforum/6_merging/monthly_stuff.csv') # date,price,hashrate,coins_per_block,efficiency,max_efficiency
monthly_price_joules["real_joules_per_coin"] = monthly_price_joules["hashrate"] / (monthly_price_joules["efficiency"] * monthly_price_joules["coins_per_block"])

price_plot_x = pd.to_datetime(monthly_price_joules['date'])
price_plot_y = process(monthly_price_joules['price'])

joules_plot_x = pd.to_datetime(monthly_price_joules['date'])
joules_plot_y = process(monthly_price_joules['real_joules_per_coin'])



price_offset = 0
price_plot_y = price_plot_y + price_offset

joules_offset = 0
joules_plot_y = joules_plot_y + joules_offset

time_to_double = 2.6
# koomey's law: after 2018, the efficiency doubles every 2.6 years
first_2018_efficiency = quarterly_data[quarterly_data['quarter'] == '2017-12-31']['TH/J'].values[0] # 2018-01-01
koomey_preds_efficiency = []
for i in range(4*(2024-2018)+1):
    koomey_preds_efficiency.append(first_2018_efficiency * 2**((i/4)/time_to_double))
koomey_preds_efficiency = np.array(koomey_preds_efficiency)
koomey_preds_efficiency = process(koomey_preds_efficiency,apply_multiplier=True)
koomey_preds_x = pd.date_range(start='2018-01-01', periods=len(koomey_preds_efficiency), freq='Q')

# koomey's law for joules:
first_2018_joules = monthly_price_joules[monthly_price_joules['date'] == '2017-12-31']['real_joules_per_coin'].values[0]
koomey_preds_joules = []
for i in range(12*(2024-2018)+1):
    koomey_preds_joules.append(first_2018_joules * 2**((i/12)/time_to_double))
koomey_preds_joules = np.array(koomey_preds_joules)
koomey_preds_joules = process(koomey_preds_joules)
koomey_preds_joules_x = pd.date_range(start='2018-01-01', periods=len(koomey_preds_joules), freq='M')

# koomey's law for price:
first_2018_price = monthly_price_joules[monthly_price_joules['date'] == '2017-12-31']['price'].values[0]
koomey_preds_price = []
for i in range(12*(2024-2018)+1):
    koomey_preds_price.append(first_2018_price * 2**((i/12)/time_to_double))
koomey_preds_price = np.array(koomey_preds_price)
koomey_preds_price = process(koomey_preds_price)
koomey_preds_price_x = pd.date_range(start='2018-01-01', periods=len(koomey_preds_price), freq='M')


In [ ]:
yearly_counts = gray_scatter_x.dt.year.value_counts().sort_index()
print(yearly_counts.mean(),yearly_counts.std())
# export to latex
print(yearly_counts.to_frame().T.to_latex())

In [ ]:
plt.tight_layout()
plt.rcParams['font.family'] = 'Times New Roman'

plt.rcParams.update({'font.size': 15})

koomey_extra = " starting 2018"
koomey_extra = ""

# Step 4: Plot single point for each quarter + all the data points using the best model
plt.figure(figsize=(12, 8), dpi=dpi)
plt.scatter(gray_scatter_x, gray_scatter_y, c='gray', s=5, label='BitcoinTalk.org data collected with LLM', alpha=0.5)
plt.scatter(lime_scatter_x, lime_scatter_y, c='lime', s=50, label=f'Data monthly averaged {func}({unit})', alpha=0.5)
plt.scatter(red_scatter_x, red_scatter_y, c='red', s=50, label=f'Data quarterly averaged {func}({unit})', alpha=0.5)
plt.plot(max_plot_x, max_plot_y, color='gray', label=func+'($P_{max}$)', linewidth=2)
plt.plot(price_plot_x, price_plot_y, color='blue', label=func+'(Price)'+("" if price_offset == 0 else str(price_offset)), linewidth=2)
plt.plot(joules_plot_x, joules_plot_y, color='green', label=func+'(Joules per coin)'+("" if joules_offset == 0 else str(joules_offset)), linewidth=2)
plt.plot(koomey_preds_x, koomey_preds_efficiency, color='purple', label='Koomey\'s Law for efficiency' + koomey_extra, linewidth=2)
plt.plot(koomey_preds_joules_x, koomey_preds_joules, color='orange', label='Koomey\'s Law applied to joules per coin' + koomey_extra, linewidth=2)
plt.plot(koomey_preds_price_x, koomey_preds_price, color='brown', label='Koomey\'s Law applied to price' + koomey_extra, linewidth=2)
# plt.plot(quarterly_data['quarter'][1:], np.log(best_model_fit.fittedvalues[1:]), label='Model: ln($\hat{P}_{eff}$)', linestyle='--')
plt.xlabel('Time')
plt.ylabel(func+'($\hat{P}_{eff}$) (' + unit + ') for efficiency. price and joules are ' + func + '(\$ or J)')
plt.yscale('linear')
# put legend in bottom right
plt.legend(loc='lower right')
plt.grid(which='major', linestyle='-', linewidth='0.5', color='black')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.minorticks_on()
# Start the x axis at 2011
plt.xlim(pd.Timestamp('2010-11-01'), pd.Timestamp('2024-01-01'))
# Start the y axis at -1
plt.ylim(-4, 11)

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# Adjust the position of the box to be next to the plot
plt.savefig('pdfs/data_plot.pdf', format='pdf', bbox_inches='tight')
plt.show()


In [ ]:

def plot_efficiency_comparison(ax):
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['mathtext.fontset'] = 'stix'


    # Plot efficiency and max efficiency
    ax.plot(red_scatter_x, red_scatter_y, label='Empirical average', color='blue', linewidth=0.8)
    ax.plot(max_plot_x, max_plot_y, label='Maximum attainable', color='black', linewidth=0.8, linestyle='--')


    # Fill the area, Interpolate red_scatter_y to match the length of max_plot_x
    interp2 = np.interp(max_plot_x, red_scatter_x, red_scatter_y)
    ax.fill_between(max_plot_x, max_plot_y, interp2, interpolate=True, color='lightblue', alpha=0.5)



    # Set x-ticks to every year and format them to include "01.01."
    start = pd.Timestamp('2010-12-31 00:00:00')
    end = pd.Timestamp('2024-01-31 00:00:00')
    years = pd.date_range(start=start, end=end, freq='YS')
    ax.set_xticks(years)
    ax.set_xticklabels([f"{year.year}" for year in years], rotation=30)
    # set xlim
    ax.set_xlim(start, end)

    # Set y-axis label and format y-ticks to display as 10^x
    ax.set_ylabel('Power efficiency (TH/J)', rotation='vertical')
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: "$10^{" + str(x-12).replace(".0","") + "}$")) #.rstrip(".0")

    # Set font to Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'

    # Make the font larger
    fs = 15
    plt.rcParams.update({'font.size': fs})

    # Increase DPI
    plt.rcParams.update({'figure.dpi': 100})

    # Rotate x-ticks
    plt.xticks(rotation=30)

    # Add a grid
    # Add more gridlines
    ax.grid(which='major', color='#DDDDDD', linewidth=0.8)
    ax.grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.8)
    ax.minorticks_on()

    # ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=len(colors) + 2, fancybox=True, shadow=True)
    # ax.legend(loc='upper left', bbox_to_anchor=(0, 0.9), ncol=1, fancybox=True, shadow=True)
    ax.legend()



plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(figsize=(12, 6))
plot_efficiency_comparison(ax)
plt.savefig('pdfs/efficiencycomparison_wide.pdf', format='pdf', bbox_inches='tight')
plt.show()
fig, ax = plt.subplots(figsize=(6, 6))
plot_efficiency_comparison(ax)
plt.savefig('pdfs/efficiencycomparison_thin.pdf', format='pdf', bbox_inches='tight')
plt.show()

# Old IQR plots (not used)

In [ ]:
quarterly_dates = red_scatter_x
data = pd.DataFrame({'Date': gray_scatter_x, 'Performance': gray_scatter_y})
quarterly_data2 = pd.DataFrame({'Date': red_scatter_x, 'Performance': red_scatter_y})
# Function to calculate KDE, extract mode, IQR, and max performance
def analyze_performance(data, bandwidth=0.3):
    kde = gaussian_kde(data['Performance'], bw_method=bandwidth)
    performance_range = np.linspace(data['Performance'].min(), data['Performance'].max(), 500)
    kde_values = kde(performance_range)
    mode = performance_range[np.argmax(kde_values)]
    iqr = np.subtract(*np.percentile(data['Performance'], [75, 25]))
    max_performance = data['Performance'].max()
    return mode, iqr, max_performance

# Analyzing each quarter
results = []
for date in red_scatter_x:
    quarter_start = date - pd.offsets.QuarterBegin(startingMonth=1)
    quarter_end = date
    quarter_data = data[(data['Date'] >= quarter_start) & (data['Date'] <= quarter_end)]
    if not quarter_data.empty:
        mode, iqr, max_performance = analyze_performance(quarter_data)
        results.append({'Date': date, 'Mode': mode, 'IQR': iqr, 'Max': max_performance, 'Max-Mode': max_performance - mode})

results_df = pd.DataFrame(results)

# Plotting
plt.figure(figsize=(12, 8))
plt.scatter(data['Date'], data['Performance'], c='gray', s=5, label='Individual Data Points', alpha=0.5)
plt.scatter(quarterly_data2['Date'], quarterly_data2['Performance'], c='red', s=50, label='Quarterly Averages', alpha=0.5)
plt.plot(results_df['Date'], results_df['Mode'], label='Mode of KDE', color='blue', linewidth=2)
plt.fill_between(results_df['Date'], results_df['Mode'] - results_df['IQR'], results_df['Mode'] + results_df['IQR'], color='blue', alpha=0.2, label='IQR')
plt.plot(results_df['Date'], results_df['Max-Mode'], label='Max - Mode', color='green', linestyle='--', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Performance (log10 hashes per joule)')
plt.legend()
plt.grid(True)
plt.show()

# Plotting the IQR width as a function of time
plt.subplot(2, 1, 2)
plt.plot(results_df['Date'], results_df['IQR'], label='IQR Width', color='purple', marker='x')
plt.xlabel('Time')
plt.ylabel('IQR Width')
plt.title('IQR Width as a Function of Time')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()